# LAB 04 - Data preprocessing 2

## 01 데이터 구조 점검 1



<연습 문제 1>
고객 데이터와 구매 데이터를 통합하여, 연령대별 평균 구매액 분석하기
- 팀장님께서 고객 정보와 최근 구매 내역 데이터를 전달해주셨습니다.
- 하지만 데이터가 완벽하지는 않네요.
- 여러분의 데이터 분석 스킬을 발휘하여 아래 요구사항에 따라 데이터를 정제하고 분석 리포트를 만들어 주세요.


<요구 사항>
- 구매 데이터의 price 컬럼에 있는 결측치는 전체 상품의 평균 가격으로 채워주세요.

- price가 1000을 초과하는 데이터는 이상치로 간주하고, 전체 상품의 중앙값으로 대체해주세요.

- 정제된 두 데이터프레임을 customer_id 기준으로 병합하세요.

- 최종 병합된 데이터프레임에 대한 기술 통계량을 도출하세요.

In [1]:
from hossam import load_data
import numpy as np
from pandas import DataFrame
from pandas import merge
from pandas import CategoricalDtype
from sklearn.impute import SimpleImputer

In [2]:
# 1. 데이터 로드

order_history = load_data('order_history')
order_history

[data] https://data.hossam.kr/data/lab04/order_history.xlsx
[desc] 신입 데이터 분석가를 위한 실전 미션!

field        description
-----------  -------------
purchase_id  일련번호
customer_id  고객번호
item         상품명
price        금액



,customer_id,item,price
purchase_id,,,
1,101,Laptop,1200.0
2,102,Mouse,25.0
3,103,Keyboard,NaN
4,101,Monitor,300.0
5,104,Webcam,5000.0
6,102,USB Hub,30.0
7,105,Laptop,1100.0
8,103,Mouse,NaN
9,101,External HDD,80.0


In [3]:
customer_info = DataFrame({'customer_id':[101,102,103,104,105,106],'name':['Alice','Bob','Charlie','David','Eve','Frank'],'age':[25,32,45,38,29,41]})
customer_info

,customer_id,name,age
0,101,Alice,25
1,102,Bob,32
2,103,Charlie,45
3,104,David,38
4,105,Eve,29
5,106,Frank,41


In [4]:
# 2. 요구사항 처리 (1) 구매 데이터의 price 컬럼에 있는 결측치는 전체 상품의 평균 가격으로 채워주세요.


#결측치 제거 규칙을 담은 객체 생성
imr = SimpleImputer(missing_values=np.nan,strategy='mean')
imr

,missing_values,nan
,strategy,'mean'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False


In [5]:
#생성된 규칙 활용해서 결측치 처리
order_history_copy = order_history.copy()

df_imr = imr.fit_transform(order_history_copy[['price']])
df_imr

array([[1200.   ],
       [  25.   ],
       [ 985.625],
       [ 300.   ],
       [5000.   ],
       [  30.   ],
       [1100.   ],
       [ 985.625],
       [  80.   ],
       [ 150.   ]])

In [6]:
#결측치가 처리된 열을 다시 복사본에 넣어주기
order_history_copy['price'] = df_imr
order_history_copy

,customer_id,item,price
purchase_id,,,
1,101,Laptop,1200.000
2,102,Mouse,25.000
3,103,Keyboard,985.625
4,101,Monitor,300.000
5,104,Webcam,5000.000
6,102,USB Hub,30.000
7,105,Laptop,1100.000
8,103,Mouse,985.625
9,101,External HDD,80.000


In [7]:
# 2. 요구사항 처리 (2) price가 1000을 초과하는 데이터는 이상치로 간주하고, 전체 상품의 중앙값으로 대체해주세요.

order_history_copy2 = order_history_copy.copy()


order_history_copy2.loc[order_history_copy2['price']>1000,'price']=np.nan
order_history_copy2



,customer_id,item,price
purchase_id,,,
1,101,Laptop,NaN
2,102,Mouse,25.000
3,103,Keyboard,985.625
4,101,Monitor,300.000
5,104,Webcam,NaN
6,102,USB Hub,30.000
7,105,Laptop,NaN
8,103,Mouse,985.625
9,101,External HDD,80.000


In [8]:
#중앙값으로 대체
imr2 = SimpleImputer(missing_values=np.nan,strategy='median')
imr2

,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False


In [9]:
#생성된 규칙 활용해서 결측치 처리
order_history_copy3 = order_history_copy2.copy()

df_imr2 = imr2.fit_transform(order_history_copy3[['price']])
df_imr2

array([[150.   ],
       [ 25.   ],
       [985.625],
       [300.   ],
       [150.   ],
       [ 30.   ],
       [150.   ],
       [985.625],
       [ 80.   ],
       [150.   ]])

In [10]:
#결측치가 처리된 열을 다시 복사본에 넣어주기
order_history_copy3['price'] = df_imr2
order_history_copy3

,customer_id,item,price
purchase_id,,,
1,101,Laptop,150.000
2,102,Mouse,25.000
3,103,Keyboard,985.625
4,101,Monitor,300.000
5,104,Webcam,150.000
6,102,USB Hub,30.000
7,105,Laptop,150.000
8,103,Mouse,985.625
9,101,External HDD,80.000


In [11]:
# 2. 요구사항 처리 (3) 정제된 두 데이터프레임을 customer_id 기준으로 병합하세요.

merged_df = merge(order_history_copy3,customer_info)
merged_df

,customer_id,item,price,name,age
0,101,Laptop,150.000,Alice,25
1,102,Mouse,25.000,Bob,32
2,103,Keyboard,985.625,Charlie,45
3,101,Monitor,300.000,Alice,25
4,104,Webcam,150.000,David,38
5,102,USB Hub,30.000,Bob,32
6,105,Laptop,150.000,Eve,29
7,103,Mouse,985.625,Charlie,45
8,101,External HDD,80.000,Alice,25
9,106,Docking Station,150.000,Frank,41


In [ ]:
# 2. 요구사항 처리 (4) 최종 병합된 데이터프레임에 대한 기술 통계량을 도출하세요.

#str 값인 name 와 age 는 자동적으로 제외되었지만, 숫자값이나 명목형이어야하는 customer_id 는 타입 변환을 해주고 기술통계량을 구해야 좀 더 정확함
merged_df.describe()

,customer_id,price,age
count,10.00000,10.000000,10.00000
mean,102.80000,300.625000,33.70000
std,1.75119,369.246135,8.04225
min,101.00000,25.000000,25.00000
25%,101.25000,97.500000,26.00000
50%,102.50000,150.000000,32.00000
75%,103.75000,262.500000,40.25000
max,106.00000,985.625000,45.00000


In [ ]:
#price 와 age 에 대해서만 기술 통계량 정리
type_change = merged_df.astype({'customer_id':'str'})
type_change.describe()

,price,age
count,10.000000,10.00000
mean,300.625000,33.70000
std,369.246135,8.04225
min,25.000000,25.00000
25%,97.500000,26.00000
50%,150.000000,32.00000
75%,262.500000,40.25000
max,985.625000,45.00000


### 결측치 처리 다른 접근 방식

<평균값을 직접 구해서 fillna 를 활용>
- 결측치처리 = purchases_df.copy()
- mean_price = 결측치처리['price'].mean()
- 결측치처리['price'] = 결측치처리['price'].fillna(mean_price)
- 결측치처리



### 이상치 처리 다른 접근 방식

<1000이 넘는 값을 결측치로 만들어서 없앨필요 없이, 1000이 넘는 경우에 중앙값을 넣어주는 접근>
- 이상치처리 = 결측치처리.copy()
- median_price = 이상치처리['price'].median()
- 이상치처리.loc[이상치처리['price'] > 1000, 'price'] = median_price
- 이상치처리


